In [ ]:
import sys
sys.path.append("..")

import openagent
from openagent.llms._openai import OpenAI as guidance_llm

from dotenv import load_dotenv
load_dotenv()

llmcc = guidance_llm(
    model="gpt-3.5-turbo"
)
llmtc = guidance_llm(
    model="text-davinci-003"
)

In [ ]:
chat_template = '''
                {{#user~}}
                I want to translate the following English text into Python code:
                QUERY: {{input}}
                {{~/user}}

                {{#assistant~}}
                Sure, I can assist with that. If I need more information, I'll ask for clarification.
                {{~/assistant}}

                {{#user~}}
                Yes, go ahead and write the complete code.
                {{~/user}}

                {{#assistant~}}
                {{gen 'response' temperature=0 max_tokens=3900}}
                {{~/assistant}}

                {{#assistant~}}
                If the context or the task is not clear, please provide additional information to clarify.
                {{~/assistant}}'''

In [ ]:
from openagent.agent.chat import ChatAgent

agent = ChatAgent(
    llm=llmcc,
    prompt_template=chat_template,
    )

In [ ]:
# gpt_prompt = """
# We are writing snake in python. MVC components split in separate files. Keyboard control.
# Make food pellet orange. Always have 2 food pellets on the screen at any time.
# Make pellets move, by changing position every 5 seconds in any of the 8 directions by 1 cell.
# Make the game grid 50 by 50 cells.
# Display score in the upper right corner. Snake eating 1 pellet adds 100 to the score.
# """

gpt_prompt = """Write a simple python program to get fibonacci of 20."""
# gpt_prompt = """Generate Python code to calculate the Fibonacci sequence up to 20 terms and return a formatted string representation of the sequence."""

res = agent.run(input=gpt_prompt)

In [ ]:
print(res)

In [ ]:
c = f"""{res.split('```')[1].replace('python', '')}"""
# print(c)
# print(type(c))
c

In [ ]:
# Split the code into lines
lines = c.split('\n')

# Find the common leading whitespace
common_whitespace = min(len(line) - len(line.lstrip()) for line in lines if line.strip())

# Remove the common whitespace from each line
formatted_code = '\n'.join(line[common_whitespace:] for line in lines)

# Print or use the formatted_code as needed
print(formatted_code)
formatted_code


In [2]:
%run agentbox.py --gpt_prompt "calculate my age using python program. Give answer in year, months, and days. Use my date of birth as 12/11/1997."


from datetime import date

def calculate_age(birth_date):
    today = date.today()
    age = today.year - birth_date.year
    if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
        age -= 1
    months = (today.month - birth_date.month) % 12
    days = (today.day - birth_date.day) % 30  # Assuming each month has 30 days

    return age, months, days

# Enter your date of birth in the format: dd/mm/yyyy
birth_date = date(1998, 8, 16)
age_years, age_months, age_days = calculate_age(birth_date)

print("Your age is: {} years, {} months, and {} days.".format(age_years, age_months, age_days))



'Your age is: 25 years, 0 months, and 1 days.'

Code execution completed


In [3]:
import ast
import subprocess
import sys

def install_dependencies(code):
    try:
        # Parse the code to extract import statements
        parsed_ast = ast.parse(code)
        imports = []

        for node in ast.walk(parsed_ast):
            if isinstance(node, ast.Import):
                imports.extend([name.name for name in node.names])
            elif isinstance(node, ast.ImportFrom):
                module_name = node.module
                if module_name is not None:
                    imports.append(module_name.split('.')[0])

        # Remove duplicate imports and filter out standard library modules
        imports = list(set(imports))
        third_party_dependencies = [dep for dep in imports if dep not in sys.modules]
        print(imports, third_party_dependencies)

        if third_party_dependencies:
            subprocess.check_call([sys.executable, "-m", "pip", "install"] + third_party_dependencies)
            return True
        else:
            print("No third-party dependencies detected.")
            return True
    except subprocess.CalledProcessError:
        print("Dependency installation failed.")
        return False

# Example usage
code = """
from datetime import date
import requests
import numpy
"""

install_dependencies(code)


['datetime', 'requests', 'numpy'] ['requests', 'numpy']


True

In [4]:
code = "from datetime import date\nimport requests\nimport numpy\nfrom urllib.request import urlopen\nfrom requests.exceptions import Timeout"
install_dependencies(code)

['datetime', 'requests', 'numpy', 'urllib'] ['requests', 'numpy']


True

In [17]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import sys
sys.path.append("..")
import openagent
from openagent.llms._openai import OpenAI as guidance_llm
from openagent.agent.chat import ChatAgent
from dotenv import load_dotenv
load_dotenv()

from jupyter_client import KernelManager
from IPython import display
import subprocess
import ast
import time
import argparse
import textwrap
import threading


def install_dependencies(code):
    try:
        # Parse the code to extract import statements
        parsed_ast = ast.parse(code)
        imports = [node.names[0].name.split('.')[0] for node in ast.walk(parsed_ast) if isinstance(node, ast.Import) or isinstance(node, ast.ImportFrom)]
        
        # print(imports)
        if imports:
            subprocess.check_call(["pip", "install"] + imports)
            return True
        else:
            print("No dependencies detected.")
            return True
    except subprocess.CalledProcessError:
        print("Dependency installation failed.")
        return True # Change this to False


def run_python_code_in_kernel(code):
    # Create a kernel manager
    km = KernelManager(kernel_name='python3')  # Use the appropriate kernel name

    # Start the kernel
    km.start_kernel()

    # Connect to the kernel
    kc = km.client()
    kc.start_channels()

    # Execute the code in the kernel
    kc.execute(code)

    # Create a thread for waiting on messages
    def wait_for_messages():
        try:
            while True:
                msg = kc.get_iopub_msg()
                msg_type = msg['header']['msg_type']
                print("msg ", msg)
                print("msg_type", msg_type)
                if msg_type == 'display_data':
                    output_data = msg['content']['data']
                    if 'image/png' in output_data:
                        display.display_png(output_data['image/png'], raw=True)
                    elif 'image/jpeg' in output_data:
                        display.display_jpeg(output_data['image/png'], raw=True)
                elif msg_type == 'stream':
                    output_data = msg['content']['text']
                    output_data = output_data.split("\n")
                    for output in output_data[:-1]:
                        display.display(output)
        except asyncio.CancelledError:
            pass  # Ignore the exception
    
    # Start the message-waiting thread
    message_thread = threading.Thread(target=wait_for_messages)
    message_thread.start()

    # Wait for the specified timeout
    timeout_seconds = 5
    message_thread.join(timeout_seconds)

    # Check if the thread is still alive (indicating timeout)
    if message_thread.is_alive():
        print("Code execution completed")
    else:
        print("Code execution completed within the timeout.")

    # Stop the kernel
    kc.stop_channels()
    km.shutdown_kernel()

# Main function
def main():
    code = """
            from datetime import date
            def calculate_age(birth_date):
                today = date.today()
                age = today.year - birth_date.year
                if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
                    age -= 1
                months = (today.month - birth_date.month) % 12
                days = (today.day - birth_date.day) % 30  # Assuming each month has 30 days

                return age, months, days

            # Enter your date of birth in the format: dd/mm/yyyy
            birth_date = date(1998, 4, 4)
            age_years, age_months, age_days = calculate_age(birth_date)

            print("Your age is: {} years, {} months, and {} days.".format(age_years, age_months, age_days))
        """
    code = textwrap.dedent(code)
    # agent = agent()
    # res = agent().run(input=gpt_prompt)
    # code = f"""{res.split('```')[1].replace('python', '')}"""
    # print(code)

    # Install dependencies
    if install_dependencies(code):
        # Run the generated code in the Jupyter kernel
        run_python_code_in_kernel(code)

if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description='Execute Python code from the command line.')
    # parser.add_argument("--gpt_prompt", help="Python code to be executed", default=None)
    # args = parser.parse_args()
    # gpt_prompt = args.gpt_prompt

    main()


Dependency installation failed.
msg  {'header': {'msg_id': '21a8f5c9-876b28498418cb09c8d8296b_22336_1', 'msg_type': 'status', 'username': 'username', 'session': '21a8f5c9-876b28498418cb09c8d8296b', 'date': datetime.datetime(2023, 8, 17, 6, 50, 7, 683511, tzinfo=tzutc()), 'version': '5.3'}, 'msg_id': '21a8f5c9-876b28498418cb09c8d8296b_22336_1', 'msg_type': 'status', 'parent_header': {'msg_id': '8c13aafd-b9e370c8fee7c47b36dddc88_14776_0', 'msg_type': 'execute_request', 'username': 'username', 'session': '8c13aafd-b9e370c8fee7c47b36dddc88', 'date': datetime.datetime(2023, 8, 17, 6, 50, 5, 636362, tzinfo=tzutc()), 'version': '5.3'}, 'metadata': {}, 'content': {'execution_state': 'busy'}, 'buffers': []}
msg_type status
msg  {'header': {'msg_id': '21a8f5c9-876b28498418cb09c8d8296b_22336_2', 'msg_type': 'execute_input', 'username': 'username', 'session': '21a8f5c9-876b28498418cb09c8d8296b', 'date': datetime.datetime(2023, 8, 17, 6, 50, 7, 684510, tzinfo=tzutc()), 'version': '5.3'}, 'msg_id': 

'Your age is: 25 years, 4 months, and 13 days.'

msg  {'header': {'msg_id': '21a8f5c9-876b28498418cb09c8d8296b_22336_5', 'msg_type': 'status', 'username': 'username', 'session': '21a8f5c9-876b28498418cb09c8d8296b', 'date': datetime.datetime(2023, 8, 17, 6, 50, 7, 712463, tzinfo=tzutc()), 'version': '5.3'}, 'msg_id': '21a8f5c9-876b28498418cb09c8d8296b_22336_5', 'msg_type': 'status', 'parent_header': {'msg_id': '8c13aafd-b9e370c8fee7c47b36dddc88_14776_0', 'msg_type': 'execute_request', 'username': 'username', 'session': '8c13aafd-b9e370c8fee7c47b36dddc88', 'date': datetime.datetime(2023, 8, 17, 6, 50, 5, 636362, tzinfo=tzutc()), 'version': '5.3'}, 'metadata': {}, 'content': {'execution_state': 'idle'}, 'buffers': []}
msg_type status
Code execution completed
